In [ ]:
import pdfplumber

with pdfplumber.open("Financial Terms.pdf") as pdf:
    first_page = pdf.pages[0]
    print(first_page.chars)

In [ ]:
term_report = pdfplumber.open("Financial Terms.pdf").pages
fin_term_en_cn = {}
for page in term_report:
    tmp_page= page.extract_text(x_tolerance=1, y_tolerance=1).split('\n')
    for term in tmp_page:
        term = term.split()
        if len(term)<2:
            print(term)
        else:
            fin_term_en_cn[term[0]] = ' '.join(term[1:])

In [30]:
import json 
with open('fin_term_cn_2_en.json','w') as f:
    json.dump(fin_term_en_cn,f, indent=3)

In [1]:
import pdfplumber
term_report = pdfplumber.open("Q3_BAIDU_2018.pdf").pages

In [51]:
import re
def date_parser(text):
    month_list = [
        'January',
        'February',
        'March',
        'April',
        'May',
        'June',
        'July',
        'August',
        'September',
        'October',
        'November',
        'December'
    ]
    month_res = re.findall('|'.join([month+' \d{1,2}' for month in month_list]), text)
    year_res = re.findall('19\d{2}|20\d{2}',text)
    if len(month_res) > len(year_res):
        year_res.extend(year_res[-1]*(len(month_res)-len(year_res)))
    res = [', '.join([m, y]) for m,y in zip(month_res, year_res)]
    return res


In [82]:
def balance_sheet(page):
    text = page.extract_text(x_tolerance=1, y_tolerance=1)
    data_list = date_parser(text)
    text_list = text.split('\n')
    for i, line in enumerate(text_list):
        line = line.split('  ')
        line = [element for element in line if len(element)>1]
        text_list[i] = line
    print(text_list)
    res = {}
    scope = None
    sub_scope = None
    
    propertie = ['assets', 'liabilities', 'equity']
    state = ['current', 'non-current']
    is_total = 'total' 
    trigger_words = ['assets', 'liabilities and equity']
    
    for line in text_list:
        if len(line) == 1 and line[0].lower() in trigger_words:
            scope = line[0]
            res[scope] = {}
        elif scope and len(line) == 1:
            line[0] = line[0].replace(':','')
            line_list = line[0].split()
            print(line_list)
            if line_list[0].lower() in state and line_list[1].lower() in propertie:
                sub_scope = line[0]
                res[scope][sub_scope] = {}
        elif scope and sub_scope:
            print(line)
            print(scope)
            print(sub_scope)
            if len(data_list) > len(line) - 1:
                line.extend([0]*(len(data_list)-len(line) + 1))
            first_word_list = line[0].split()
            if first_word_list[0].lower() == is_total and first_word_list[1].lower() in state:
                sub_scope = None
                
            res[scope][sub_scope][line[0]] = {date:num for date, num in zip(data_list,line[1:])}
            
    return res

In [83]:
balance_sheet(term_report[10])

[[], ['Baidu, Inc. '], ['Condensed Consolidated Balance Sheets '], ['December 31,', 'September 30,'], ['(In RMB millions except for share information)', ' 2017', '2018'], ['Audited', 'Unaudited'], ['ASSETS'], ['Current assets:'], ['Cash and cash equivalents', ' 11,084', '15,775'], ['Restricted cash', ' 252', '15'], ['Short-term investments', ' 89,381', '88,716'], ['Other invested securities', '18,350'], ['Accounts receivable, net', ' 4,571', '5,455'], ['Loans and interest receivable, net', '23,938'], ['Amounts due from related parties', ' 168', '23,321'], ['Other assets, current', ' 3,425', '6,700'], ['Total current assets', ' 151,169', '139,982'], [], ['Non-current assets:'], ['Fixed assets, net', ' 12,475', '16,197'], ['Intangible assets, net', ' 5,467', '8,732'], ['Goodwill', ' 15,806', '17,051'], ['Long-term investments, net', ' 56,283', '80,589'], ['Loans and interest receivable, net', '3,467'], ['Amounts due from related parties', ' 9', '1,994'], ['Deferred tax assets, net', ' 1,

KeyError: None